In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install tensorflow-gpu==2.0.0-alpha
!pip install split-folders

     |████████████████████████████████| 332.1MB 47kB/s 
     |████████████████████████████████| 3.0MB 39.5MB/s 
     |████████████████████████████████| 419kB 48.7MB/s 


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import split_folders
import numpy as np
import cv2
import os



/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [0]:
real = "/content/drive/My Drive/real-and-fake-face-detection/real_and_fake_face/training_real"
fake = "/content/drive/My Drive/real-and-fake-face-detection/real_and_fake_face/training_fake"

real_path = os.listdir(real)
fake_path = os.listdir(fake)

In [0]:
def load_img(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (224, 224))
    return image[...,::-1]

In [0]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=False,
                                   rescale=1./255,
                                   )

In [0]:
dataset_path = "/content/drive/My Drive/real-and-fake-face-detection/real_and_fake_face"

In [9]:
train = train_datagen.flow_from_directory(dataset_path,
                                          class_mode="binary",
                                          target_size=(96, 96),
                                          batch_size=32)

Found 2051 images belonging to 2 classes.


In [0]:
x, y = next(train)

0.0

In [10]:
ResNet = ResNet50(input_shape=(96, 96, 3),
                          include_top=False,
                          weights='imagenet'
                          )

average_layer = GlobalAveragePooling2D()

model = Sequential([
    ResNet,
    average_layer,
    Dense(256, activation=tf.nn.relu),
    BatchNormalization(),
    Dropout(0.2),
    Dense(2, activation=tf.nn.softmax)
])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 3, 3, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               524544    
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 256)               1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 514       
Total params: 24,113,794
Trainable params: 24,060,162
Non-trainable params: 53,632
_____________________________________

In [0]:
model.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [0]:
# from keras.models import load_model
# model = load_model('model.h5')

In [13]:
model.fit_generator(train,
                    epochs=50,
                    callbacks=[lr_callbacks])

Epoch 1/50
65/65 [==============================] - 791s 12s/step - loss: 0.9067 - accuracy: 0.5695
Epoch 2/50
65/65 [==============================] - 463s 7s/step - loss: 0.7532 - accuracy: 0.5929
Epoch 3/50
65/65 [==============================] - 468s 7s/step - loss: 0.7176 - accuracy: 0.5982
Epoch 4/50
65/65 [==============================] - 470s 7s/step - loss: 0.6942 - accuracy: 0.5929
Epoch 5/50
65/65 [==============================] - 467s 7s/step - loss: 0.6997 - accuracy: 0.6046
Epoch 6/50
65/65 [==============================] - 466s 7s/step - loss: 0.6622 - accuracy: 0.6363
Epoch 7/50
65/65 [==============================] - 468s 7s/step - loss: 0.6702 - accuracy: 0.6319
Epoch 8/50
65/65 [==============================] - 466s 7s/step - loss: 0.6473 - accuracy: 0.6602
Epoch 9/50
65/65 [==============================] - 465s 7s/step - loss: 0.6219 - accuracy: 0.6743
Epoch 10/50
65/65 [==============================] - 464s 7s/step - loss: 0.6236 - accuracy: 0.6845
Epoch 11

In [0]:
model.save("model.h5")


In [15]:
model.evaluate_generator(train)

[0.01888733204597464, 0.9970746]

In [0]:
model.predict_generator(train)

array([[1.9152135e-03, 9.9808478e-01],
       [9.9932277e-01, 6.7729759e-04],
       [9.9961483e-01, 3.8509612e-04],
       ...,
       [9.9999952e-01, 5.2479760e-07],
       [9.0291673e-05, 9.9990976e-01],
       [9.9930334e-01, 6.9661456e-04]], dtype=float32)